#  Transformation部件传递

## 源码分析

### 类的定义

In [ ]:
class TransformChain(Chain):
    """Chain that transforms the chain output.

    Example:
        .. code-block:: python

            from langchain.chains import TransformChain
            transform_chain = TransformChain(input_variables=["text"],
             output_variables["entities"], transform=func())
    """

`TransformChain` 是什么？
- `TransformChain` 是一个继承自 `Chain` 的类，主要目的是在链式调用中对数据进行转换处理。
- 它的基本功能是：接受一些输入，经过一个转换函数（同步或异步）处理后，输出新的数据。
- 这个类特别适合用在需要对数据进行某种转换或处理的场景下，作为其他链的辅助部分。

### 属性定义

In [ ]:
    input_variables: List[str]
    """The keys expected by the transform's input dictionary."""
    
    output_variables: List[str]
    """The keys returned by the transform's output dictionary."""
    
    transform_cb: Callable[[Dict[str, str]], Dict[str, str]] = Field(alias="transform")
    """The transform function."""
    
    atransform_cb: Optional[Callable[[Dict[str, Any]], Awaitable[Dict[str, Any]]]] = (
        Field(None, alias="atransform")
    )
    """The async coroutine transform function."""

`input_variables`
- 定义：`input_variables` 是一个字符串列表，表示 `TransformChain` 期望从输入字典中接收哪些键。
- 用途：在链式调用中，输入的数据通常是一个字典。`input_variables` 用来指定这个字典中哪些键是 `TransformChain` 需要处理的。比如，如果 `input_variables` 包含 `"text"`，那么输入字典中必须有一个 `"text"` 键，其对应的值将被传递给 `TransformChain` 进行处理。

`output_variables`
- 定义：`output_variables` 是一个字符串列表，表示 `TransformChain` 将返回哪些键。
- 用途：这告诉调用者，`TransformChain` 将会输出一个字典，这个字典包含的键是 `output_variables` 中指定的键。比如，如果 `output_variables` 包含 `"entities"`，那么 `TransformChain` 的输出字典中将有一个 `"entities"` 键。

`transform_cb`
- 定义：`transform_cb` 是一个同步的回调函数，用于将输入字典转换为输出字典。
- 类型：它是一个接受 `Dict[str, str]` 类型的输入并返回 `Dict[str, str]` 类型输出的函数。这个函数由用户提供，用于对输入数据进行处理。
- 用途：`transform_cb` 是 `TransformChain` 的核心功能，它实现了输入到输出的转换逻辑。

`atransform_cb`
- 定义：`atransform_cb` 是一个可选的异步回调函数，用于异步转换输入字典为输出字典。
- 类型：它是一个接受 `Dict[str, Any]` 类型的输入并返回 `Awaitable[Dict[str, Any]]` 类型输出的异步函数。
- 用途：在需要异步处理的场景中，`atransform_cb` 提供了异步版本的转换逻辑。如果没有提供异步函数，代码会回退到同步的 `transform_cb`。

### 静态方法`_log_once`

In [1]:
    @staticmethod
    @functools.lru_cache
    def _log_once(msg: str) -> None:
        """Log a message once.

        :meta private:
        """
        logger.warning(msg)

- 功能：这个方法用来记录一次性警告消息，确保同样的警告不会重复记录多次。
- 实现：通过 `functools.lru_cache` 装饰器，确保相同的消息只会被记录一次。这在性能上比较友好，避免了冗余日志。
- 用途：当 `atransform_cb` 未提供时，代码会回退到同步的 `transform_cb` 并记录一次警告消息。

### 属性方法 `input_keys` 和 `output_keys`

In [ ]:
    @property
    def input_keys(self) -> List[str]:
        """Expect input keys.

        :meta private:
        """
        return self.input_variables

    @property
    def output_keys(self) -> List[str]:
        """Return output keys.

        :meta private:
        """
        return self.output_variables

- `input_keys`：
  - 返回 `input_variables`，即该链期望的输入键。这些键是在链的执行过程中从输入数据字典中提取出来的。
- `output_keys`：
  - 返回 `output_variables`，即该链期望的输出键。这些键是链在执行完转换函数后返回的数据字典中的键。

### 同步方法`_call`

In [ ]:
    def _call(
        self,
        inputs: Dict[str, str],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        return self.transform_cb(inputs)

- 功能：这是链的同步调用方法。
- 参数：
  - `inputs`: 一个字典，包含链的输入数据。字典的键应与 `input_variables` 中定义的键一致。
  - `run_manager`: 可选的回调管理器，用于处理链执行期间的回调事件。
- 实现：直接调用 `transform_cb`（同步转换函数）处理输入数据，并返回转换后的结果。

### 异步方法`_acall`

In [2]:
    async def _acall(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[AsyncCallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        if self.atransform_cb is not None:
            return await self.atransform_cb(inputs)
        else:
            self._log_once(
                "TransformChain's atransform is not provided, falling"
                " back to synchronous transform"
            )
            return self.transform_cb(inputs)

- 功能：这是链的异步调用方法。
- 参数：
  - `inputs`: 一个字典，包含链的输入数据。字典的键应与 `input_variables` 中定义的键一致。
  - `run_manager`: 可选的异步回调管理器，用于处理链执行期间的异步回调事件。
- 实现：
  - 如果 `atransform_cb`（异步转换函数）存在，则异步调用它处理输入数据，并返回转换后的结果。
  - 如果 `atransform_cb` 不存在，则记录一次警告消息，并回退到调用同步的 `transform_cb` 处理输入数据。

## demo

In [11]:
from langchain.chains.transform import TransformChain
from typing import Dict
from langchain.chains.sequential import SimpleSequentialChain

# 第一步：提取所有大写字母
def extract_uppercase_letters(inputs: Dict[str, str]) -> Dict[str, str]:
    text = inputs["text"]
    uppercase_letters = ''.join([char for char in text if char.isupper()])
    return {"uppercase_letters": uppercase_letters}

# 第二步：移除所有元音字母（针对小写和大写）
def remove_vowels(inputs: Dict[str, str]) -> Dict[str, str]:
    text = inputs["text"]
    vowels = "AEIOUaeiou"
    text_without_vowels = ''.join([char for char in text if char not in vowels])
    return {"text_without_vowels": text_without_vowels}

# 第三步：逆序排列字符
def reverse_text(inputs: Dict[str, str]) -> Dict[str, str]:
    text = inputs["text"]
    reversed_text = text[::-1]
    return {"reversed_text": reversed_text}

# 创建 TransformChain 实例
uppercase_chain = TransformChain(
    input_variables=["text"],
    output_variables=["uppercase_letters"],
    transform=extract_uppercase_letters
)

remove_vowels_chain = TransformChain(
    input_variables=["text"],
    output_variables=["text_without_vowels"],
    transform=remove_vowels
)

reverse_chain = TransformChain(
    input_variables=["text"],
    output_variables=["reversed_text"],
    transform=reverse_text
)

# 将多个链连接成一个顺序链
sequential_chain = SimpleSequentialChain(
    chains=[uppercase_chain, remove_vowels_chain, reverse_chain],
    input_key="text",
    output_key="final_output"
)

# 使用顺序链处理输入数据
input_data = {"text": "Hello HHHWorasfdsafld! This is a Test."}
output_data = sequential_chain.run(input_data)

# 输出结果
print(output_data)  # 输出: {'final_output': 'stT s sThT'}


TTWHHHH
